# A-B-C Credit Evaluation System - Analysis Notebook

This notebook demonstrates how to use the credit evaluation system for loan risk assessment.

## Overview
- **A (Low Risk)**: Score 0-300 - Excellent creditworthiness
- **B (Medium Risk)**: Score 301-600 - Moderate risk profile
- **C (High Risk)**: Score 601-1000 - Elevated default risk

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import our credit evaluation system
from credit_evaluator import CreditEvaluator
from data_processor import DataProcessor
from risk_classifier import RiskClassifier

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ Libraries imported successfully!")

## 1. Load and Explore Data

In [ ]:
# Initialize data processor
processor = DataProcessor()

# Load sample data (replace with your actual data loading)
df = processor.load_sample_data(n_samples=2000)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nDefault rate: {df['default_risk'].mean():.2%}")

# Display first few rows
df.head()

In [ ]:
# Data exploration
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

# Plot key features
features_to_plot = ['age', 'annual_income', 'credit_score', 'debt_to_income', 'credit_utilization', 'payment_history_score']

for i, feature in enumerate(features_to_plot):
    axes[i].hist(df[feature], bins=30, alpha=0.7)
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 2. Train Credit Evaluation Models

In [ ]:
# Prepare data
X = df.drop('default_risk', axis=1)
y = df['default_risk']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution:")
print(y.value_counts(normalize=True))

In [ ]:
# Train different models and compare performance
models = ['logistic_regression', 'random_forest', 'xgboost', 'lightgbm']
results = {}

for model_type in models:
    print(f"\nTraining {model_type}...")
    
    # Initialize evaluator
    evaluator = CreditEvaluator(model_type=model_type)
    
    # Train model
    metrics = evaluator.fit(X, y, validation_split=0.2)
    
    if metrics:
        results[model_type] = {
            'ROC AUC': metrics['roc_auc'],
            'Accuracy': metrics['accuracy'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1_score']
        }
        print(f"✅ {model_type}: ROC AUC = {metrics['roc_auc']:.4f}")

# Display results
results_df = pd.DataFrame(results).T
print("\n📊 Model Comparison:")
print(results_df.round(4))

In [ ]:
# Plot model comparison
fig, ax = plt.subplots(figsize=(12, 6))
results_df.plot(kind='bar', ax=ax)
plt.title('Model Performance Comparison')
plt.xlabel('Models')
plt.ylabel('Score')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Find best model
best_model = results_df['ROC AUC'].idxmax()
print(f"🏆 Best model: {best_model} (ROC AUC: {results_df.loc[best_model, 'ROC AUC']:.4f})")

## 3. Use Best Model for Risk Assessment

In [ ]:
# Initialize with best performing model
final_evaluator = CreditEvaluator(model_type=best_model)
final_evaluator.fit(X, y, validation_split=0.2)

print(f"Final model trained: {best_model}")

In [ ]:
# Create test clients for demonstration
test_clients = pd.DataFrame({
    'age': [25, 45, 35, 55, 30],
    'annual_income': [35000, 85000, 60000, 120000, 45000],
    'credit_score': [580, 750, 680, 820, 620],
    'debt_to_income': [0.6, 0.2, 0.4, 0.15, 0.5],
    'credit_history_length': [2, 15, 8, 25, 5],
    'num_credit_accounts': [3, 8, 5, 12, 4],
    'credit_utilization': [0.9, 0.3, 0.6, 0.1, 0.8],
    'payment_history_score': [0.7, 0.95, 0.85, 0.98, 0.75],
    'num_late_payments': [5, 0, 2, 0, 4],
    'employment_length': [1, 10, 6, 20, 3],
    'home_ownership': ['rent', 'own', 'mortgage', 'own', 'rent'],
    'loan_purpose': ['debt_consolidation', 'home_improvement', 'major_purchase', 'other', 'debt_consolidation']
})

client_ids = [f"CLIENT_{i+1:03d}" for i in range(len(test_clients))]

print("Test clients created:")
test_clients

In [ ]:
# Get risk predictions
risk_scores, risk_categories = final_evaluator.predict_risk(test_clients)

# Create results dataframe
results_df = test_clients.copy()
results_df['client_id'] = client_ids
results_df['risk_score'] = risk_scores
results_df['risk_category'] = risk_categories

# Display results
print("🎯 Risk Assessment Results:")
display_cols = ['client_id', 'age', 'credit_score', 'debt_to_income', 'risk_score', 'risk_category']
results_df[display_cols]

In [ ]:
# Generate detailed client reports
print("📋 Detailed Client Reports:\n")

for i, client_id in enumerate(client_ids):
    client_report = final_evaluator.generate_client_report(
        test_clients.iloc[i], client_id
    )
    
    print(f"--- {client_report['client_id']} ---")
    print(f"Risk Score: {client_report['risk_score']:.0f}")
    print(f"Risk Category: {client_report['risk_category']} ({client_report['risk_level']})")
    print(f"Default Probability: {client_report['default_probability']}")
    print(f"Recommendation: {client_report['recommendation']}")
    print(f"Interest Rate Modifier: +{client_report['interest_rate_modifier']:.1f}%")
    print(f"Monitoring: {client_report['monitoring_frequency']}")
    print()

## 4. Portfolio Analysis

In [ ]:
# Portfolio analysis
portfolio_results = final_evaluator.evaluate_portfolio(test_clients, client_ids=client_ids)
portfolio_summary = portfolio_results['portfolio_summary']

print("📊 Portfolio Analysis:")
print(f"Total Clients: {portfolio_summary['total_clients']}")
print(f"Average Risk Score: {portfolio_summary['average_risk_score']:.0f}")
print(f"Portfolio Risk Level: {portfolio_summary['portfolio_risk_level']}")
print(f"\nRisk Distribution:")
print(f"  - A (Low Risk): {portfolio_summary['risk_distribution']['A_count']} clients ({portfolio_summary['risk_distribution']['A_percentage']:.1f}%)")
print(f"  - B (Medium Risk): {portfolio_summary['risk_distribution']['B_count']} clients ({portfolio_summary['risk_distribution']['B_percentage']:.1f}%)")
print(f"  - C (High Risk): {portfolio_summary['risk_distribution']['C_count']} clients ({portfolio_summary['risk_distribution']['C_percentage']:.1f}%)")

In [ ]:
# Visualize portfolio
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Risk score distribution
axes[0].hist(risk_scores, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Risk Score Distribution')
axes[0].set_xlabel('Risk Score')
axes[0].set_ylabel('Frequency')
axes[0].axvline(x=300, color='green', linestyle='--', alpha=0.7, label='A-B Threshold')
axes[0].axvline(x=600, color='orange', linestyle='--', alpha=0.7, label='B-C Threshold')
axes[0].legend()

# Risk category pie chart
category_counts = pd.Series(risk_categories).value_counts()
colors = {'A': 'lightgreen', 'B': 'orange', 'C': 'lightcoral'}
pie_colors = [colors.get(cat, 'gray') for cat in category_counts.index]
axes[1].pie(category_counts.values, labels=[f"{cat} ({count})" for cat, count in category_counts.items()], 
           autopct='%1.1f%%', colors=pie_colors)
axes[1].set_title('Risk Category Distribution')

# Risk score vs client
colors_scatter = [colors.get(cat, 'gray') for cat in risk_categories]
axes[2].scatter(range(len(risk_scores)), risk_scores, c=colors_scatter, alpha=0.7, s=100)
axes[2].set_title('Risk Scores by Client')
axes[2].set_xlabel('Client Index')
axes[2].set_ylabel('Risk Score')
axes[2].axhline(y=300, color='green', linestyle='--', alpha=0.7, label='A-B Threshold')
axes[2].axhline(y=600, color='orange', linestyle='--', alpha=0.7, label='B-C Threshold')
axes[2].legend()

plt.tight_layout()
plt.show()

## 5. Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = final_evaluator.get_feature_importance()

if feature_importance is not None:
    print("🔍 Top 10 Most Important Features:")
    print(feature_importance.head(10))
    
    # Plot feature importance
    plt.figure(figsize=(10, 8))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Top 15 Feature Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("Feature importance not available for this model type")

## 6. Model Persistence

In [ ]:
# Save the trained model
model_path = 'trained_credit_model.pkl'
final_evaluator.save_model(model_path)

print(f"✅ Model saved to {model_path}")
print("\nYou can now load this model in production using:")
print("evaluator = CreditEvaluator()")
print("evaluator.load_model('trained_credit_model.pkl')")

## 7. Next Steps for Production

### Immediate Actions:
1. **Replace sample data** with your actual credit data
2. **Adjust risk thresholds** in `config.py` based on your business requirements
3. **Fine-tune model hyperparameters** for better performance

### Integration:
1. **API Integration**: Create REST API endpoints for real-time scoring
2. **Database Integration**: Connect to your customer database
3. **Monitoring**: Set up model performance monitoring
4. **Retraining**: Implement automated model retraining pipeline

### Compliance:
1. **Regulatory Review**: Ensure model meets regulatory requirements
2. **Bias Testing**: Test for fairness across different demographic groups
3. **Documentation**: Document model decisions for audit trails
4. **Validation**: Implement ongoing model validation procedures